In [3]:
from zeno_client import ZenoClient, ZenoMetric
import pandas as pd
import json
import numpy as np

In [4]:
client = ZenoClient("zen_kLh235pvRwxLNZctmZiUcVJc4VOXfMdXdrXZwHZdp4I")

## From CSV

In [6]:
df_gpt4 = pd.read_csv("results/for_analysis/gpt/gpt4_zeno.csv")

## CUTOFF

In [160]:
file_path = 'results/for_analysis/gpt/gpt4_coh.json'

In [161]:
with open(file_path, 'r') as file:
            data = json.load(file)
            df = pd.DataFrame(data)
df.head()

,doc_id,system_id,source,reference,system_output,scores,prompt,all_responses
0,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M11,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson was brought on with only seven min...,"{'coherence': 1.3333333333333333, 'consistency...",You will be given one summary written for a ne...,"[Coherence: 1, Coherence: 2, Coherence: 2, Coh..."
1,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M13,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 2.3333333333333335, 'consistency...",You will be given one summary written for a ne...,"[Coherence: 3, Coherence: 3, Coherence: 3, Coh..."
2,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M1,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 2.3333333333333335, 'consistency...",You will be given one summary written for a ne...,"[Coherence: 4, Coherence: 4, Coherence: 4, Coh..."
3,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M14,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 1.6666666666666667, 'consistency...",You will be given one summary written for a ne...,"[Coherence: 3, Coherence: 3, Coherence: 2, Coh..."
4,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M15,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 3.3333333333333335, 'consistency...",You will be given one summary written for a ne...,"[Coherence: 4, Coherence: 3, Coherence: 3, Coh..."


In [14]:
df_init = df[['source', 'system_output', 'scores']]

In [25]:
df.loc[:, "id"] = df.index.astype(str)

In [23]:
df_init['data'] = 'Source : \n' + df_init['source'] + '\n\nSummary : \n' + df_init['system_output']

/var/folders/jm/2zytm77j0ms47q8tymwmv8800000gn/T/ipykernel_32290/2098530224.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_init['data'] = 'Source : \n' + df_init['source'] + '\n\nSummary : \n' + df_init['system_output']


In [ ]:
df_init.drop(columns=['source', 'system_output'], inplace=True)

In [121]:
project = client.create_project(
    name="SummEval",
    view="text-classification",
    metrics=[
        ZenoMetric(name="mean_diff_coherence", type="mean", columns=["coh_diff"]),
        ZenoMetric(name="mean_diff_fluency", type="mean", columns=["flu_diff"]),
        ZenoMetric(name="mean_diff_consistency", type="mean", columns=["con_diff"]),
        ZenoMetric(name="mean_diff_relevance", type="mean", columns=["rel_diff"]),
        ZenoMetric(name="mean_diff_all", type="mean", columns=["mean_diff"]),
    ]
)

project.upload_dataset(df_init, id_column="id", data_column='data', label_column="scores")

Successfully updated project.
Access your project at  https://hub.zenoml.com/project/6dda2a1e-e3e1-4b0a-8b31-0e2b3b19b1cf/SummEval


  0%|          | 0/5 [00:00<?, ?it/s]

Successfully uploaded data


In [27]:
df_coherence = df[['all_responses']]

In [58]:
df_coherence.loc[:, "id"] = df_coherence.index.astype(str)

In [57]:
df_coherence.loc[:, 'prob_score'] = df['all_responses'].apply(
        lambda x: np.mean([int(i[-1]) for i in x if i and i[-1].isdigit()]))

In [60]:
# df_coherence.loc[:, 'coherence'] = df_coherence.apply(
#     lambda row: int(row['prob_score'] == df_init.loc[row.name, 'scores']['coherence']),
#     axis=1
# )

In [79]:
df_coherence.loc[:, 'coh_diff'] = df_coherence.apply(
    lambda row: float(row['prob_score'] - df_init.loc[row.name, 'scores']['coherence']),
    axis=1
)

/var/folders/jm/2zytm77j0ms47q8tymwmv8800000gn/T/ipykernel_32290/3443648651.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.13333333  0.66666667  1.11666667 ... -0.05        0.66666667
  0.03333333]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_coherence.loc[:, 'coh_diff'] = df_coherence.apply(


In [83]:
df_coherence.drop('coherence', axis=1, inplace=True)

/var/folders/jm/2zytm77j0ms47q8tymwmv8800000gn/T/ipykernel_32290/2288720778.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coherence.drop('coherence', axis=1, inplace=True)


In [84]:
project.upload_system(df_coherence, name="GPT4-coherence", id_column="id", output_column="prob_score")

  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded system


In [102]:
project.delete_system("GPT4-coherence")

Successfully deleted system


No comment

In [128]:
df_gpt4.loc[:, 'coherence'] = df_coherence['prob_score']

### RELEVANCE

In [85]:
file_path = 'results/for_analysis/gpt/gpt4_rel.json'
with open(file_path, 'r') as file:
            relevance_data = json.load(file)
            relevance_df = pd.DataFrame(relevance_data)
relevance_df.head()

,doc_id,system_id,source,reference,system_output,scores,prompt,all_responses
0,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M11,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson was brought on with only seven min...,"{'coherence': 1.3333333333333333, 'consistency...",You will be given one summary written for a ne...,"[Relevance: 1, Relevance: 2, Relevance: 1, Rel..."
1,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M13,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 2.3333333333333335, 'consistency...",You will be given one summary written for a ne...,"[Relevance: 4, Relevance: 4, Relevance: 5, Rel..."
2,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M1,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 2.3333333333333335, 'consistency...",You will be given one summary written for a ne...,"[Relevance: 4, Relevance: 4, Relevance: 4, Rel..."
3,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M14,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 1.6666666666666667, 'consistency...",You will be given one summary written for a ne...,"[Relevance: 3, Relevance: 4, Relevance: 4, Rel..."
4,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M15,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 3.3333333333333335, 'consistency...",You will be given one summary written for a ne...,"[Relevance: 5, Relevance: 4, Relevance: 5, Rel..."


In [163]:
identical_doc = (relevance_df['doc_id'] == df['doc_id']).all()
identical_doc

True

In [91]:
df_gpt4 = df_coherence.copy()

In [140]:
relevance_df.loc[:, 'prob_score'] = relevance_df['all_responses'].apply(
        lambda x: np.mean([int(i[-1]) for i in x if i and i[-1].isdigit()]))

In [141]:
df_gpt4.loc[:, 'relevance'] = relevance_df['prob_score']

In [96]:
df_gpt4['output'] = 'Coherence: ' + df_gpt4['prob_score'].astype(str) + '\nRelevance: ' + df_gpt4['relevance'].astype(str)

In [144]:
# df_gpt4.loc[:, 'coherence'] = df_gpt4['prob_score']
# df_gpt4.drop(columns=['prob_score'],axis=1, inplace=True)
df_gpt4.loc[:, 'rel_diff'] = relevance_df.apply(
    lambda row: float(row['prob_score'] - df_init.loc[row.name, 'scores']['relevance']),
    axis=1
)

In [100]:
df_gpt4.drop(columns=['all_responses'],axis=1, inplace=True)

In [109]:
project.upload_system(df_gpt4, name="GPT4", id_column="id", output_column="output")

  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded system


In [105]:
df_gpt4.head()

,id,coh_diff,relevance,output,coherence,rel_diff
0,0,-0.133333,1.20,Coherence: 1.2\nRelevance: 1.2,1.20,-0.466667
1,1,0.666667,3.00,Coherence: 3.0\nRelevance: 3.0,3.00,0.333333
2,2,1.116667,3.45,Coherence: 3.45\nRelevance: 3.45,3.45,0.783333
3,3,0.933333,2.60,Coherence: 2.6\nRelevance: 2.6,2.60,-0.066667
4,4,0.266667,3.60,Coherence: 3.6\nRelevance: 3.6,3.60,-0.400000


In [108]:
df_init.head()

,scores,id,data
0,"{'coherence': 1.3333333333333333, 'consistency...",0,Source : \nPaul Merson has restarted his row w...
1,"{'coherence': 2.3333333333333335, 'consistency...",1,Source : \nPaul Merson has restarted his row w...
2,"{'coherence': 2.3333333333333335, 'consistency...",2,Source : \nPaul Merson has restarted his row w...
3,"{'coherence': 1.6666666666666667, 'consistency...",3,Source : \nPaul Merson has restarted his row w...
4,"{'coherence': 3.3333333333333335, 'consistency...",4,Source : \nPaul Merson has restarted his row w...


### CONSISTENCY

In [134]:
file_path = 'results/for_analysis/gpt/gpt4_con.json'
with open(file_path, 'r') as file:
            consistency_data = json.load(file)
            consistency_df = pd.DataFrame(consistency_data)
consistency_df.head()

,doc_id,system_id,source,reference,system_output,scores,prompt,all_responses
0,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M11,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson was brought on with only seven min...,"{'coherence': 1.3333333333333333, 'consistency...",You will be given a news article. You will the...,"[Consistency: 1, Consistency: 1, Consistency: ..."
1,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M13,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 2.3333333333333335, 'consistency...",You will be given a news article. You will the...,"[Consistency: 5, Consistency: 5, Consistency: ..."
2,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M1,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 2.3333333333333335, 'consistency...",You will be given a news article. You will the...,"[Consistency: 5, Consistency: 5, Consistency: ..."
3,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M14,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 1.6666666666666667, 'consistency...",You will be given a news article. You will the...,"[Consistency: 5, Consistency: 5, Consistency: ..."
4,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M15,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 3.3333333333333335, 'consistency...",You will be given a news article. You will the...,"[Consistency: 5, Consistency: 5, Consistency: ..."


In [159]:
identical_doc = (relevance_df['doc_id'] == consistency_df['doc_id']).all()
identical_doc

True

In [138]:
consistency_df.loc[:, 'prob_score'] = consistency_df['all_responses'].apply(
        lambda x: np.mean([int(i[-1]) for i in x if i and i[-1].isdigit()]))

In [137]:
consistency_df.head()

,doc_id,system_id,source,reference,system_output,scores,prompt,all_responses,prob_score
0,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M11,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson was brought on with only seven min...,"{'coherence': 1.3333333333333333, 'consistency...",You will be given a news article. You will the...,"[Consistency: 1, Consistency: 1, Consistency: ...",1.20
1,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M13,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 2.3333333333333335, 'consistency...",You will be given a news article. You will the...,"[Consistency: 5, Consistency: 5, Consistency: ...",3.00
2,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M1,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 2.3333333333333335, 'consistency...",You will be given a news article. You will the...,"[Consistency: 5, Consistency: 5, Consistency: ...",3.45
3,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M14,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 1.6666666666666667, 'consistency...",You will be given a news article. You will the...,"[Consistency: 5, Consistency: 5, Consistency: ...",2.60
4,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M15,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 3.3333333333333335, 'consistency...",You will be given a news article. You will the...,"[Consistency: 5, Consistency: 5, Consistency: ...",3.60


In [139]:
df_gpt4.loc[:, 'consistency'] = consistency_df['prob_score']

In [142]:
df_gpt4['output'] = 'Coherence: ' + df_gpt4['coherence'].astype(str) + '\nRelevance: ' + df_gpt4['relevance'].astype(str) + '\nConsistency: ' + df_gpt4['consistency'].astype(str)

In [143]:
df_gpt4.loc[:, 'con_diff'] = consistency_df.apply(
    lambda row: float(row['prob_score'] - df_init.loc[row.name, 'scores']['consistency']),
    axis=1
)

In [175]:
project.upload_system(df_gpt4, name="GPT4", id_column="id", output_column="output")

  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded system


In [148]:
consistency_df.loc[:, 'consistency'] = consistency_df.apply(
    lambda row: int(row['prob_score'] == df_init.loc[row.name, 'scores']['consistency']),
    axis=1
)

In [149]:
consistency_df['consistency'].value_counts()

consistency
1    1126
0     474
Name: count, dtype: int64

### FLUENCY

In [150]:
file_path = 'results/for_analysis/gpt/gpt4_flu.json'
with open(file_path, 'r') as file:
            fluency_data = json.load(file)
            fluency_df = pd.DataFrame(fluency_data)
fluency_df.head()

,doc_id,system_id,source,reference,system_output,scores,prompt,all_responses
0,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M11,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson was brought on with only seven min...,"{'coherence': 1.3333333333333333, 'consistency...",You will be given one summary written for a ne...,"[Fluency: 2, Fluency: 2, Fluency: 2, Fluency: ..."
1,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M13,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 2.3333333333333335, 'consistency...",You will be given one summary written for a ne...,"[Fluency: 4, Fluency: 4, Fluency: 4, Fluency: ..."
2,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M1,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 2.3333333333333335, 'consistency...",You will be given one summary written for a ne...,"[Fluency: 4, Fluency: 4, Fluency: 3, Fluency: ..."
3,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M14,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 1.6666666666666667, 'consistency...",You will be given one summary written for a ne...,"[Fluency: 4, Fluency: 4, Fluency: 4, Fluency: ..."
4,dm-test-8764fb95bfad8ee849274873a92fb8d6b400eee2,M15,Paul Merson has restarted his row with Andros ...,Andros Townsend an 83rd minute sub in Tottenha...,Paul merson has restarted his row with andros ...,"{'coherence': 3.3333333333333335, 'consistency...",You will be given one summary written for a ne...,"[Fluency: 3, Fluency: 3, Fluency: 3, Fluency: ..."


In [167]:
identical_doc = (fluency_df['system_id'] == df['system_id']).all()
identical_doc

True

In [168]:
fluency_df.loc[:, 'prob_score'] = fluency_df['all_responses'].apply(
        lambda x: np.mean([int(i[-1]) for i in x if i and i[-1].isdigit()]))

In [169]:
df_gpt4.loc[:, 'fluency'] = fluency_df['prob_score']

In [170]:
df_gpt4['output'] = 'Coherence: ' + df_gpt4['coherence'].astype(str) + '\nRelevance: ' + df_gpt4['relevance'].astype(str) + '\nConsistency: ' + df_gpt4['consistency'].astype(str) + '\nFluency: ' + df_gpt4['fluency'].astype(str)

In [171]:
df_gpt4.loc[:, 'flu_diff'] = fluency_df.apply(
    lambda row: float(row['prob_score'] - df_init.loc[row.name, 'scores']['fluency']),
    axis=1
)

In [187]:
project.upload_system(df_gpt4, name="GPT4", id_column="id", output_column="output")

  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded system


Mean diff & upload df

In [174]:
df_gpt4.loc[:, 'mean_diff'] = df_gpt4[['coh_diff', 'flu_diff', 'con_diff', 'rel_diff']].mean(axis=1)

In [183]:
df_gpt4.loc[:, 'mean'] = df_gpt4[['coherence', 'relevance', 'consistency', 'fluency']].mean(axis=1) 

In [185]:
df_gpt4['output'] = 'Coherence: ' + df_gpt4['coherence'].astype(str) + '\nConsistency: ' + df_gpt4['consistency'].astype(str) + '\nFluency: ' + df_gpt4['fluency'].astype(str) + '\nRelevance: ' + df_gpt4['relevance'].astype(str) + '\nOverall:' + df_gpt4['mean'].astype(str)

In [186]:
df_gpt4.to_csv('results/for_analysis/gpt/gpt4_zeno.csv', index=False)

Output improving

In [177]:
df_init.to_csv('results/for_analysis/zeno_init.csv', index=False)

In [178]:
def format_label(label_dict):
    return "\n".join([f"{key.capitalize()}: {value}" for key, value in label_dict.items()])

In [179]:
df_init['scores'] = df_init['scores'].apply(format_label)

/var/folders/jm/2zytm77j0ms47q8tymwmv8800000gn/T/ipykernel_32290/1499980976.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_init['scores'] = df_init['scores'].apply(format_label)


In [180]:
df_init.head()

,scores,id,data
0,Coherence: 1.3333333333333333\nConsistency: 1....,0,Source : \nPaul Merson has restarted his row w...
1,Coherence: 2.3333333333333335\nConsistency: 5....,1,Source : \nPaul Merson has restarted his row w...
2,Coherence: 2.3333333333333335\nConsistency: 5....,2,Source : \nPaul Merson has restarted his row w...
3,Coherence: 1.6666666666666667\nConsistency: 5....,3,Source : \nPaul Merson has restarted his row w...
4,Coherence: 3.3333333333333335\nConsistency: 5....,4,Source : \nPaul Merson has restarted his row w...


In [181]:
project = client.create_project(
    name="SummEval",
    view="text-classification",
    metrics=[
        ZenoMetric(name="mean_diff_coherence", type="mean", columns=["coh_diff"]),
        ZenoMetric(name="mean_diff_fluency", type="mean", columns=["flu_diff"]),
        ZenoMetric(name="mean_diff_consistency", type="mean", columns=["con_diff"]),
        ZenoMetric(name="mean_diff_relevance", type="mean", columns=["rel_diff"]),
        ZenoMetric(name="mean_diff_all", type="mean", columns=["mean_diff"]),
    ]
)

project.upload_dataset(df_init, id_column="id", data_column='data', label_column="scores")

Successfully updated project.
Access your project at  https://hub.zenoml.com/project/6dda2a1e-e3e1-4b0a-8b31-0e2b3b19b1cf/SummEval


  0%|          | 0/5 [00:00<?, ?it/s]

Successfully uploaded data


In [2]:
df_gpt4

NameError: name 'df_gpt4' is not defined